In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_parquet("Задание/train_data.pqt")
test_df = pd.read_parquet("Задание/test_data.pqt")

In [10]:
dic = {'object': [],
      'nums': []}
for i in col:
    if df[i].dtype.name != 'object':
        dic['nums'].append(i)
    else:
        dic['object'].append(i)
dic

{'object': ['start_cluster',
  'channel_code',
  'city_type',
  'ogrn_month',
  'ogrn_year',
  'okved',
  'segment'],
 'nums': ['balance_amt_avg',
  'ft_registration_date',
  'max_founderpres',
  'sum_of_paym_2m',
  'sum_of_paym_1y',
  'sum_a_oper_1m',
  'sum_b_oper_1m',
  'sum_c_oper_1m',
  'sum_deb_d_oper_1m',
  'sum_cred_d_oper_1m',
  'sum_deb_e_oper_1m',
  'sum_cred_e_oper_1m',
  'sum_deb_f_oper_1m',
  'sum_cred_f_oper_1m',
  'sum_deb_g_oper_1m',
  'sum_cred_g_oper_1m',
  'sum_deb_h_oper_1m',
  'sum_cred_h_oper_1m',
  'sum_a_oper_3m',
  'sum_b_oper_3m',
  'sum_c_oper_3m',
  'sum_deb_d_oper_3m',
  'sum_cred_d_oper_3m',
  'sum_deb_e_oper_3m',
  'sum_cred_e_oper_3m',
  'sum_deb_f_oper_3m',
  'sum_cred_f_oper_3m',
  'sum_deb_g_oper_3m',
  'sum_cred_g_oper_3m',
  'sum_deb_h_oper_3m',
  'sum_cred_h_oper_3m']}

Записаны метрики, с которыми вышли лучшие результаты

Пишем модель с разными входными параметрами 

In [3]:
# Выбираем метрики для модели
col = ['start_cluster', 'balance_amt_avg', 'channel_code', 'city_type', 'ogrn_month', 'ogrn_year', 'ft_registration_date', 'max_founderpres', 'okved', 'segment', 'sum_of_paym_2m', 'sum_of_paym_1y', 'sum_a_oper_1m', 'sum_b_oper_1m', 'sum_c_oper_1m', 'sum_deb_d_oper_1m', 'sum_cred_d_oper_1m', 'sum_deb_e_oper_1m', 'sum_cred_e_oper_1m', 'sum_deb_f_oper_1m', 'sum_cred_f_oper_1m', 'sum_deb_g_oper_1m', 'sum_cred_g_oper_1m', 'sum_deb_h_oper_1m', 'sum_cred_h_oper_1m', 'sum_a_oper_3m', 'sum_b_oper_3m', 'sum_c_oper_3m', 'sum_deb_d_oper_3m', 'sum_cred_d_oper_3m', 'sum_deb_e_oper_3m', 'sum_cred_e_oper_3m', 'sum_deb_f_oper_3m', 'sum_cred_f_oper_3m', 'sum_deb_g_oper_3m', 'sum_cred_g_oper_3m', 'sum_deb_h_oper_3m', 'sum_cred_h_oper_3m']
# col = ['start_cluster'] # Тестовая (чем меньше метрик, тем быстрее модель)

# Разделяем данные на признаки (X) и целевую переменную (y)
y = df['end_cluster']  # целевая переменная

X = pd.DataFrame()
for i in col:
    if df[i].dtype.name != 'object':
        X[i]=df[i].copy()
        X.loc[X[i].isna(), i]=X[i].median()
    else:
        X[i]=pd.factorize(df[i])[0]
        
        
# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаем и Обучаем модель случайного леса
rf = RandomForestClassifier()

# Перебираем разные данные к модели
param_grid = {
    'n_estimators': [200, 300, 400],
    'max_depth': [20, 25, 30],
    'min_samples_split': [2],
    'min_samples_leaf': [1]
}

# Создаем экземпляр GridSearchCV
print('Создаем экземпляр:', end=' ')
%time model = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Обучаем модель с использованием GridSearchCV
print('Обучаем модель:', end=' ')
%time model.fit(X_train, y_train)

### Штука не быстрая, придётся подождать

# {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
# {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
# Выводим лучшие параметры
print(model.best_params_)

Создаем экземпляр: CPU times: total: 0 ns
Wall time: 0 ns
Обучаем модель: 

C:\Users\Public\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
135 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Public\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Public\anaconda\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\Public\anaconda\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Public\anaconda\Lib\site-packag

CPU times: total: 11min 41s
Wall time: 1h 41min 23s
{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [4]:
# model = RandomForestClassifier(max_depth = 10, min_samples_leaf = 1, min_samples_split = 2, n_estimators = 100)
model = RandomForestClassifier(**model.best_params_)
model.fit(X_train, y_train)


#Считаем значимость метрик
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), col[idx]])
ar_f.sort(reverse=True)

# Предсказываем значения на тестовой выборке
y_pred = model.predict(X_test)

# Оцениваем качество модели
accuracy = accuracy_score(y_test, y_pred)
ar_f

[[0.2671, 'start_cluster'],
 [0.0623, 'sum_cred_e_oper_3m'],
 [0.0549, 'sum_deb_e_oper_3m'],
 [0.0532, 'balance_amt_avg'],
 [0.0512, 'sum_deb_e_oper_1m'],
 [0.0407, 'sum_cred_e_oper_1m'],
 [0.0386, 'okved'],
 [0.032, 'ft_registration_date'],
 [0.0316, 'sum_of_paym_1y'],
 [0.03, 'sum_cred_h_oper_3m'],
 [0.0264, 'max_founderpres'],
 [0.0254, 'sum_deb_h_oper_3m'],
 [0.0248, 'sum_of_paym_2m'],
 [0.0204, 'channel_code'],
 [0.0196, 'sum_deb_g_oper_3m'],
 [0.0189, 'ogrn_month'],
 [0.0182, 'sum_deb_h_oper_1m'],
 [0.0175, 'sum_deb_g_oper_1m'],
 [0.0175, 'sum_cred_h_oper_1m'],
 [0.0162, 'ogrn_year'],
 [0.0161, 'sum_deb_f_oper_3m'],
 [0.0115, 'sum_deb_d_oper_3m'],
 [0.0112, 'sum_deb_f_oper_1m'],
 [0.0103, 'segment'],
 [0.0101, 'sum_cred_g_oper_3m'],
 [0.0089, 'sum_deb_d_oper_1m'],
 [0.0088, 'sum_b_oper_3m'],
 [0.0087, 'city_type'],
 [0.0082, 'sum_a_oper_3m'],
 [0.0076, 'sum_c_oper_3m'],
 [0.0062, 'sum_cred_g_oper_1m'],
 [0.0056, 'sum_c_oper_1m'],
 [0.0051, 'sum_a_oper_1m'],
 [0.0043, 'sum_b_oper_

Считаем точность модели

In [5]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

cluster_weights = pd.read_excel("Задание/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

y_pred_proba = model.predict_proba(X_test)
weighted_roc_auc(y_test, y_pred_proba, model.classes_, weights_dict)

0.9337127862450603

Считаем модель на новых данных

In [6]:
# Считываем данные из тестовой метрики
test_df = test_df[test_df.date == 'month_6']
test_df["start_cluster"] = df["start_cluster"].mode()[0]


X = pd.DataFrame()
for i in col:
    if df[i].dtype.name != 'object':
        X[i]=df[i].copy()
        X.loc[X[i].isna(), i]=X[i].median()
    else:
        X[i]=pd.factorize(df[i])[0]

# Предсказываем значения
model.predict(X)
test_probabilities = model.predict_proba(X)

test_probabilities

array([[3.42382083e-01, 1.06225132e-03, 5.36121319e-03, ...,
        4.85790126e-04, 6.60724878e-02, 0.00000000e+00],
       [6.28135732e-01, 4.30179563e-04, 5.77274543e-03, ...,
        8.35353408e-04, 3.78903922e-02, 0.00000000e+00],
       [3.66641843e-01, 4.13574073e-04, 5.82440764e-03, ...,
        5.79829013e-03, 7.46007732e-02, 0.00000000e+00],
       ...,
       [7.49746653e-03, 8.00228779e-01, 1.35959141e-03, ...,
        1.10800815e-03, 1.76068139e-01, 1.58174288e-04],
       [7.51342302e-03, 8.00263443e-01, 1.37668543e-03, ...,
        1.06242930e-03, 1.75967936e-01, 1.92362322e-04],
       [1.06708897e-02, 4.52558228e-01, 2.77854936e-03, ...,
        6.41277754e-04, 5.13692464e-01, 6.40466195e-04]])

In [7]:
# Создание DataFrame с вероятностями перехода в каждый кластер для каждой строчки
cluster_probabilities_df = pd.DataFrame(test_probabilities, columns=model.classes_)

# Добавление целевой переменной к DataFrame с вероятностями
cluster_probabilities_df['id'] = test_df['id']
cluster_probabilities_df = cluster_probabilities_df.dropna(subset=['id']).reset_index(drop=True) # .astype(int)
cluster_probabilities_df.to_csv('results/sample_submission.csv', encoding='utf-8', index=False)
cluster_probabilities_df.head(9)

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ},id
0,0.366642,0.000414,0.005824,0.514037,0.005490,0.000000,0.000635,0.000000,0.014658,0.000387,0.007892,0.003551,0.000071,0.000000,0.005798,0.074601,0.000000,200000.0
1,0.849161,0.001246,0.001523,0.069449,0.001896,0.006253,0.000981,0.000075,0.010275,0.019637,0.003433,0.000635,0.002128,0.000000,0.007809,0.025498,0.000000,200001.0
2,0.003978,0.028880,0.010923,0.012550,0.001583,0.000102,0.000261,0.000000,0.002880,0.005221,0.004486,0.000249,0.001142,0.000000,0.000342,0.927396,0.000006,200002.0
3,0.023534,0.438532,0.001577,0.005899,0.001206,0.000293,0.000349,0.000019,0.001416,0.009176,0.001197,0.000174,0.000988,0.000006,0.000577,0.514894,0.000163,200003.0
4,0.025023,0.015385,0.023827,0.034737,0.009075,0.000854,0.001013,0.000599,0.019613,0.015664,0.010030,0.002269,0.006770,0.000022,0.002169,0.832943,0.000008,200004.0
5,0.059161,0.223484,0.013533,0.038401,0.003398,0.002936,0.001081,0.000213,0.006174,0.093914,0.010172,0.000995,0.003560,0.000000,0.002867,0.539941,0.000171,200005.0
6,0.145756,0.003495,0.000895,0.785701,0.001622,0.000000,0.000038,0.000427,0.001940,0.000169,0.001222,0.000000,0.000019,0.000000,0.003023,0.055691,0.000000,200006.0
7,0.015607,0.161054,0.005328,0.005616,0.001159,0.000164,0.000256,0.000090,0.002540,0.008879,0.002081,0.000213,0.001309,0.000000,0.001104,0.794308,0.000292,200007.0
8,0.067856,0.016645,0.007476,0.680223,0.000587,0.000065,0.000252,0.000206,0.003268,0.001700,0.005148,0.000982,0.000476,0.000000,0.000654,0.214338,0.000125,200008.0


In [8]:
accuracy

0.7333833333333334

In [12]:
import pandas as pd

def replace_missing_with_mode(df):
    # Получаем список столбцов с пропущенными значениями
    columns_with_missing_values = df.columns[df.isnull().any()].tolist()
    
    # Заменяем пропущенные значения на моду для каждого столбца
    for column in columns_with_missing_values:
        mode_value = df[column].mode()[0]  # Получаем моду столбца
        df[column].fillna(mode_value, inplace=True)  # Заменяем пропущенные значения модой
    
    return df

# Пример использования функции
# Создаем DataFrame для демонстрации
data = {'A': [1, 2, 3, None, 5],
        'B': [None, 2, 3, 4, 5],
        'C': [1, None, 3, None, 5]}
df = pd.DataFrame(data)

# Заменяем пропущенные значения на моду
df = replace_missing_with_mode(df)

print(df)


     A    B    C
0  1.0  2.0  1.0
1  2.0  2.0  1.0
2  3.0  3.0  3.0
3  1.0  4.0  1.0
4  5.0  5.0  5.0
